In [1]:
import pandas as pd

import fastparquet

import dill

In [2]:
from pipe_utils import *

## Model loading

In [3]:
with open(full_path('model/pipe_11.pkl'), 'rb') as file:
    model = dill.load(file)
model

Pipeline(steps=[('aggregator',
                 <pipe_utils.Aggregator object at 0x0000015DD2CDCFA0>),
                ('classifier',
                 LGBMClassifier(class_weight='balanced', learning_rate=0.05,
                                max_depth=5, n_estimators=300, num_leaves=33,
                                random_state=44, reg_lambda=10))],
         verbose=True)

## Test data loading

In [4]:
df = pd.DataFrame()
for name in [f"data/test_data/test_data_{i}.pq" for i in '01']:
    print(f"Read <{name.split('/')[-1]}> ... ", end='')
    df = pd.concat((df, pd.read_parquet(full_path(name))), axis=0, ignore_index=True)
    print('done')

Read <test_data_0.pq> ... done
Read <test_data_1.pq> ... done


In [5]:
df

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,3000000,1,11,5,17,14,12,11,3,2,...,3,3,3,4,1,3,4,1,0,0
1,3000000,2,19,16,15,9,12,11,16,3,...,3,3,3,4,1,2,4,1,0,0
2,3000001,1,16,17,8,5,4,9,5,2,...,3,3,3,4,1,3,4,1,0,0
3,3000001,2,16,7,9,0,4,9,1,2,...,3,3,3,4,1,3,4,1,0,0
4,3000001,3,10,0,14,7,11,12,2,2,...,1,1,0,2,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4724596,3499999,5,11,13,2,0,1,6,7,2,...,3,3,3,4,1,3,5,1,0,0
4724597,3499999,6,2,2,1,8,8,11,8,2,...,3,3,3,4,1,2,5,1,0,1
4724598,3499999,7,9,0,9,12,1,11,6,2,...,3,3,3,4,1,3,4,1,0,0
4724599,3499999,8,19,0,17,8,14,11,9,3,...,3,3,3,4,1,2,4,1,0,1


In [6]:
result = pd.read_csv(full_path('data/test_target.csv'))
result

,id
0,3000000
1,3000001
2,3000002
3,3000003
4,3000004
...,...
499995,3499995
499996,3499996
499997,3499997
499998,3499998


<i>сюда будем записывать предикты

## Предикт

In [7]:
result['target'] = model.predict(df)
result

Aggregate flags: ▮
Categorial encoding: ▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮
Features from enc_paym_N: ▮▮▮11▮
Aggregate numerical ▮
Adding has_loans feature
Relative OH-encoded features
Added 7 missed columns
Aggregation completed. Result shape: (500000, 385)


,id,target
0,3000000,0
1,3000001,1
2,3000002,0
3,3000003,1
4,3000004,0
...,...,...
499995,3499995,1
499996,3499996,0
499997,3499997,1
499998,3499998,0


In [8]:
result.to_csv(full_path('data/test_predict.csv'), index=False)